In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from scipy.stats import chi2_contingency
from scipy import stats 
plot_list = []

In [2]:
df_17 = pd.read_csv('../PBL/dataset/NHIS_2017_2018_100m/NHIS_OPEN_GJ_2017_100.csv', encoding='euc-kr')
df_18 = pd.read_csv('../PBL/dataset/NHIS_2017_2018_100m/NHIS_OPEN_GJ_2018_100.csv')

In [3]:
valid_17 = df_17.copy()
valid_18 = df_18.copy()

In [4]:
valid_18["음주여부"].fillna(0, inplace=True)

In [5]:
drop_list = ['기준년도', '가입자일련번호', '시도코드', '청력(좌)', '청력(우)', '시력(좌)', '시력(우)', '구강검진수검여부', '치아우식증유무', '결손치유무', '치아마모증유무', '제3대구치(사랑니)이상', '치석', '데이터공개일자', '총콜레스테롤', 'LDL콜레스테롤', 'HDL콜레스테롤', '트리글리세라이드']

In [6]:
valid_df = pd.concat([valid_17, valid_18])
valid_df.drop(columns=drop_list, inplace=True)
valid_df.dropna(inplace=True)
valid_df.reset_index(drop=True, inplace=True)

In [7]:
add_col = valid_df['식전혈당(공복혈당)'].copy()
add_col[add_col < 126] = 0
add_col[add_col >= 126] = 1
valid_df['당뇨여부'] = add_col

In [8]:
def z_score_method(df, variable_name):
    #Takes two parameters: dataframe & variable of interest as string
    columns = df.columns

    z = np.abs(stats.zscore(df))
     
    threshold = 3
    outlier = []
    index=0
    for item in range(len(columns)):
        if columns[item] == variable_name:
            index = item
    for i, v in enumerate(z[:, index]):
        if v > threshold:
            outlier.append(i)
        else:
            continue
    return outlier

In [9]:
copytest = valid_df.copy()
# copytest["식전혈당(공복혈당)"] = round(copytest["식전혈당(공복혈당)"])
outlier_z = z_score_method(copytest, "식전혈당(공복혈당)")
print(len(outlier_z))
print(len(copytest))
valid_df = copytest.drop(copytest.index[outlier_z])
print(len(valid_df))

36458
1983201
1946743


In [10]:
def cramers_V(var1,var2) :
  crosstab = np.array(pd.crosstab(var1,var2, rownames=None, colnames=None)) # Cross table building
  stat = chi2_contingency(crosstab)[0] # Keeping of the test statistic of the Chi2 test
  obs = np.sum(crosstab) # Number of observations
  mini = min(crosstab.shape)-1 # Take the minimum value between the columns and the rows of the cross table
  return np.sqrt(stat/(obs*mini))

In [11]:
cramers = cramers_V(valid_df["당뇨여부"], valid_df["흡연상태"]) 
print(round(cramers, 2))

0.06


In [12]:
cramers_yo = cramers_V(valid_df["당뇨여부"], valid_df["요단백"])
print(round(cramers_yo, 2))

0.07


In [13]:
cramers_age = cramers_V(valid_df["당뇨여부"], valid_df["연령대코드(5세단위)"])
print(round(cramers_age, 2))

[ 8  7  9 11 10 12 14  6 13  5 15 16 17 18]
0.16
